Imports

In [1]:
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject
from rasterio.mask import mask
import glob
import matplotlib.pyplot as plt

raw_dir = "../data/raw"
processed_dir = "../data/processed"
os.makedirs(processed_dir, exist_ok=True)

Helper Functions

In [2]:
def read_raster(path):
    with rasterio.open(path) as src:
        return src.read(1), src.transform, src.crs
    
def resample_to_match(src_path, target_profile):
    with rasterio.open(src_path) as src:
        data = src.read(
            out_shape=(
                src.count,
                target_profile["height"],
                target_profile["width"],
            ),
            resampling=Resampling.bilinear,
        )
        data = data[0]
        
        new_transform = rasterio.transform.from_bounds(
            *target_profile["bounds"],
            target_profile["width"],
            target_profile["height"],
        )
    
    return data, new_transform

Vegetation Indices

In [2]:
# NDVI Calculation
def compute_ndvi(nir, red):
    return (nir - red) / (nir + red + 1e-6)

# EVI Calculation
def compute_evi(nir, red, blue):
    return 2.5 * (nir - red) / (nir + 6*red - 7.5*blue + 1)


Burn Severity

In [ ]:
# NBR Calculation
def compute_nbr(nir, swir):
    return (nir - swir) / (nir + swir + 1e-6)

# dNBR Calculation
def compute_dnbr(prefire_nbr, postfire_nbr):
    return prefire_nbr - postfire_nbr
